In [1]:
from IPython.core.display import HTML
with open('../style.css', 'r') as file:
    css = file.read()
HTML(css)

# Die Onkologie-Station

Auf einer Onkologie-Station liegen fünf Patienten in nebeneinander liegenden Zimmern.
Bis auf einen  der Patienten hat jeder genau eine Zigaretten-Marke geraucht.
Der Patient, der nicht Zigarette geraucht hat, hat Pfeife geraucht.
Jeder Patient fährt genau ein Auto und ist
an genau einer Krebs-Art erkrankt.  Zusätzlich haben Sie die folgenden Informationen:
<ol>
<li> Im Zimmer neben Michael wird Camel geraucht. </li>
<li> Der Trabant-Fahrer raucht Ernte 23 und liegt im Zimmer neben dem 
      Zungen-Krebs Patienten. </li>
<li> Rolf liegt im letzten Zimmer und hat Kehlkopf-Krebs. </li>
<li> Der West-Raucher liegt im ersten Zimmer. </li>
<li> Der Mazda-Fahrer hat Zungen-Krebs und liegt neben dem Trabant-Fahrer. </li>
<li> Der Nissan-Fahrer liegt neben dem Zungen-Krebs Patient. </li>
<li> Rudolf wünscht sich Sterbe-Hilfe und liegt zwischen dem Camel-Raucher und dem Trabant-Fahrer. </li>
<li> Der Seat Fahrer hat morgen seinen letzten Geburtstag. </li>
<li> Der Luckies Raucher liegt neben dem Patienten mit Lungen-Krebs. </li>
<li> Der Camel Raucher liegt neben dem Patienten mit Darm-Krebs. </li>
<li> Der Nissan Fahrer liegt neben dem Mazda-Fahrer. </li>
<li> Der Mercedes-Fahrer raucht Pfeife und liegt neben dem Camel Raucher. </li>
<li> Jens liegt neben dem Luckies Raucher. </li>
<li> Der Hodenkrebs-Patient hat gestern seine Eier durchs Klo gespült. </li>
</ol>
Entwickeln Sie ein <em>Python</em>-Programm, das die folgenden Fragen beantwortet:
<ol> 
<li> Was raucht der Darmkrebs-Patient? </li>
<li> Was fährt Kurt für ein Auto? </li>
</ol>

## Importing the Necessary Modules

Our goal is to solve this puzzle by first coding it as a solvability problem of propositional logic and then to solve the resulting set of clauses using the algorithm of Davis and Putnam.

In [ ]:
%%capture
%run Davis-Putnam-JW.ipynb

In order to be able to transform formulas from propositional logic into sets of clauses we import the module <tt>cnf</tt> which implements the function <tt>normalize</tt> that takes a formula and transforms it into a set of clauses.

In [ ]:
%%capture
%run CNF.ipynb

In order to write formulas conveniently, we use the parser for propositional logic.

In [ ]:
import propLogParser as plp

Using the parser and the module <tt>cnf</tt> we can impement a function $\texttt{parseCNF}(s)$ that takes a string $s$ representing a formula and transforms $s$ into an equivalent set of clauses.

In [ ]:
def parseCNF(s):
    nestedTuple = plp.LogicParser(s).parse()
    Clauses     = normalize(nestedTuple)
    return Clauses

## Auxiliary Functions

In [ ]:
def atMostOne(V):
    return { frozenset({('¬',p), ('¬', q)}) for p in V
                                            for q in V 
                                            if  p != q 
           }

Given a name $f$ and an index $i \in\{1,2,3,4,5\}$, the function $\texttt{var}(i)$ creates the string 
$f\langle i \rangle$, e.g. the call <tt>var("Japanese", 2)</tt> returns the following string:

<tt>Japanese$\langle$2$\rangle$</tt>.

In [ ]:
def var(f, i):
    return f + "<" + str(i) + ">" 

In [ ]:
var("Japanese", 2)

The call $\texttt{flatten}(\texttt{LoS})$ takes list of sets $\texttt{LoS}$ and adds all the sets in this list into one big set. 

In [ ]:
def flatten(ListOfSets):
    return {x for S in ListOfSets for x in S}

In [ ]:
flatten([{1,2,3}, {3,4,5}])

A call of the form $\texttt{x}$ will return a clause that specifies that the person with property $x$ has to live in one of the houses from $1$ to $5$.

In [ ]:
def somewhere(x):
    return frozenset({ var(x, i) for i in range(1, 5+1) })

In [ ]:
somewhere("a")

Given an exclusive set of properties $S$ and a house number $i$, the function $\texttt{atMostOne}(S, i)$ returns a set of clauses that specifies that the person living in house number $i$ has at most one of the properties from the set $S$.  For example, if 
$S = \{\texttt{"Japanese"}, \texttt{"Englishman"}, \texttt{"Spaniard"}, \texttt{"Norwegian"}, \texttt{"Ukranian"}\}$, 
then $\texttt{atMostOne}(S, 3)$ specifies that the inhabitant of house number 3 has at most one of the nationalities from the set $S$.

In [ ]:
def atMostOneAt(S, i):
    return atMostOne({ var(x, i) for x in S })

In [ ]:
atMostOneAt({"A", "B", "C"}, 1)

Implement a function $\texttt{onePerHouse}(S)$ which could be called as follows:
$$\texttt{onePerHouse}(\{\texttt{"Japanese"},
       \texttt{"Englishman"}, 
       \texttt{"Spaniard"}, \texttt{"Norwegian"}, 
       \texttt{"Ukranian"}\})
$$
This function would create a set of clauses that expresses that there has to be a house where the Japanese lives, a house where the Englishman lives, a house where the Spaniard lives, a house where the Norwegian lives, and a house
where the Ukranian lives.  Furthermore, the set of clauses would create clauses that express that these five persons live in **different** houses.

When implementing this function, you should use the functions <tt>somewhere</tt> and <tt>atMostOne</tt>.

In [ ]:
def onePerRoom(S):
    return { somewhere(x) for x in S } | flatten([ atMostOneAt(S, i) for i in range(1, 5+1) ])

In [ ]:
onePerRoom({"A", "B", "C", "D", "E"})

Given to properties $a$ and $b$ the function $\texttt{sameHouse}(a, b)$ computes a set of clauses that specifies that if the inhabitant of house number $i$ has the property $a$, then he also has the property $b$ and vice versa.  For example, $\texttt{sameHouse}(\texttt{"Japanese"}, \texttt{"Dog"})$ specifies that the Japanese guy keeps a dog.

In [ ]:
def sameRoom(a, b):
    return flatten([ parseCNF(f"{var(a, i)} ↔ {var(b, i)}") for i in range(1, 5+1) ])

In [ ]:
sameRoom("Luckies", "Camel")

Given to properties $a$ and $b$ the function $\texttt{differentRoom}(a, b)$ computes a set of clauses that specifies that the inhabitants with properties $a$ and $b$ stay in different rooms. 

In [ ]:
def differentRoom(a, b):
    "your code here"

In [ ]:
differentRoom('A', 'B')

In [ ]:
def nextTo(a, b):
    Result = parseCNF(f"{var(a,1)} → {var(b,2)}")
    for i in [2, 3, 4]:
        Result |= parseCNF(f"{var(a,i)} → {var(b,i-1)} ∨ {var(b,i+1)}")
    Result |= parseCNF(f"{var(a,5)} → {var(b,4)}")
    return Result

In [ ]:
nextTo('A', 'B')

In [ ]:
Brands   = { "Camel", "Ernte", "West", "Luckies", "Pfeife" }
Cars     = { "Trabant", "Mazda", "Nissan", "Seat", "Mercedes" }
Cancers  = { "Zunge", "Kehlkopf", "Lunge", "Darm", "Hoden" }
Names    = { "Michael", "Rolf", "Rudolf", "Jens", "Kurt" }

In [ ]:
def allClauses():
    Clauses  = onePerRoom(Brands)
    Clauses |= onePerRoom(Cars)
    Clauses |= onePerRoom(Cancers)
    Clauses |= onePerRoom(Names)
    # Im Zimmer neben Michael wird Camel geraucht. 
    "your code here"
    # Der Trabant-Fahrer raucht Ernte 23 und liegt im Zimmer neben dem 
    # Zungen-Krebs Patienten. 
    "your code here"
    # Rolf liegt im letzten Zimmer und hat Kehlkopf-Krebs.
    "your code here"
    # Der West-Raucher liegt im ersten Zimmer. 
    "your code here"
    # Der Mazda-Fahrer hat Zungen-Krebs und liegt neben dem Trabant-Fahrer. 
    "your code here"
    # Der Nissan-Fahrer liegt neben dem Zungen-Krebs Patient. 
    "your code here"
    # Rudolf wünscht sich Sterbe-Hilfe und liegt zwischen dem Camel-Raucher und dem Trabant-Fahrer. 
    "your code here"
    # Der Luckies Raucher liegt neben dem Patienten mit Lungen-Krebs. 
    "your code here"
    # Der Camel Raucher liegt neben dem Patienten mit Darm-Krebs. 
    "your code here"
    # Der Nissan Fahrer liegt neben dem Mazda-Fahrer. 
    "your code here"
    # Der Mercedes-Fahrer raucht Pfeife und liegt neben dem Camel Raucher. 
    "your code here"
    # Jens liegt neben dem Luckies Raucher. 
    "your code here"
    # Der Hodenkrebs-Patient hat gestern seine Eier durchs Klo gespült.
    "your code here"

In [ ]:
Clauses = allClauses()
Clauses

In [ ]:
len(Clauses)

In [ ]:
def main():
    Clauses = allClauses()
    return solve(Clauses)

Solving the problem takes about 7 seconds on my computer.

In [ ]:
%%time
Solution = main()

In [ ]:
Solution

## Pretty Printing the Solution

In [ ]:
from IPython.display import HTML

In [ ]:
def arb(S):
    for x in S:
        return x

In [ ]:
def extractAssignment(Solution):
    Assignment = {}
    for Unit in Solution:
        Literal = arb(Unit)
        if isinstance(Literal, str):
            number = int(Literal[-2])
            name   = Literal[:-3]
            Assignment[name] = number
    return Assignment

In [ ]:
def showHTML(Solution):
    result  = '<table style="border:2px solid blue">\n'
    result += '<tr>'
    for name in ['Room', 'Brands', 'Cars', 'Cancers', 'Names']:
        result += '<th style="color:gold; background-color:blue">' + name + '</th>'
    result += '</tr>\n'
    for chair in range(1, 5+1):
        result += '<tr><td style="border:1px solid green">' + str(chair) + '</td>'
        for Class in [Brands, Cars, Cancers, Names]:
            for x in Class:
                if Solution[x] == chair:
                    result += '<td  style="border:1px solid green">' + x + '</td>'
        result += '</tr>\n'
    result += '</table>'
    display(HTML(result))

In [ ]:
showHTML(extractAssignment(Solution))

## Checking the Uniqueness of the Solution

Given a set of unit clauses $U$, the function $\texttt{checkUniqueness}(U)$ returns a clause that is the negation of the set $U$.

In [ ]:
def negateSolution(UnitClauses):
    return { complement(arb(unit)) for unit in UnitClauses }

In [ ]:
negateSolution({ frozenset({'a'}), frozenset({('¬', 'b')}) }) 

The function $\texttt{checkUniqueness}(\texttt{Solution}, \texttt{Clauses})$  takes a set of $\texttt{Clauses}$ and a $\texttt{Solution}$ for these clauses and checks, whether this is the only solution.

In [ ]:
def checkUniqueness(Solution, Clauses):
    negation = negateSolution(Solution)
    Clauses.add(frozenset(negation))
    alternative = solve(Clauses)
    if alternative == { frozenset() }:
        print("Well done: The solution is unique!")
    else:
        print("ERROR: The solution is not unique!")

In [ ]:
checkUniqueness(Solution, Clauses)